In [1]:
# Install openpyxl to open excel file
# !pip install openpyxl
# Install spacy for nlp task
#!pip install spacy
#!pip install pyLDAvis
#!python -m pip install -U gensim
#!pip install nltk

# Topic modeling of _De avond is ongemak_ by Marieke Lucas Rijneveld

In this notebook, part 1 (Deel I) of Marieke Lucas Rijneveld's novel _De avond is ongemak_ is analyzed using topic modeling. This notebook is part of a three-part analysis of the novel, which was created in the course of the master thesis ''Naar de overkant: Motieven en topics in Marieke Lucas Rijnevelds _De avond is ongemak_'', written in 2021-2022 at the University of Vienna (degree program: Nederlandistik). All comments are written in English. 

## 1. Create dataframe


### 1.1. Load the required libraries and packages

In [2]:
import os # Supports interaction with the operating system
import pandas as pd # Package to manipulate data structures
import re # Enables working with regular expressions

### 1.2. Convert text file into a data frame
In a first step the raw text file is converted into a data frame, a two-dimensional table format, for further processing and subsequent analysis.


In [3]:
# Splits .txt file into chunks (sentences) and saves them row after row in a pandas dataframe
string = [] 

with open("data/texts/De_avond_is_ongemak_Deel_I.txt", "r", encoding='utf8') as f:
    full_text = f.read()
    for l in re.split(r"(\.)", full_text):
        if l != ".":
            string.append(l + "\n")
df = pd.DataFrame(string)

In [4]:
# .head() outputs the first 5 rows of the created data frame called "df"
df.head()

,0
0,"Er staat geschreven, ‘ik maak alle dingen nieu..."
1,\n\n\t\t\tDe ijzelregen geselt bloei tot glazi...
2,\n\n\n\n\n\n\n\t\t \t\t\t-\n\n\n\n\n\n1\n\n\n\...
3,Die ochtend smeerde moeder ons een voor een i...
4,De deksel van het blik was zo vettig dat je h...


In [5]:
# Assign column name "text"
df.columns =['text']
df.head()

,text
0,"Er staat geschreven, ‘ik maak alle dingen nieu..."
1,\n\n\t\t\tDe ijzelregen geselt bloei tot glazi...
2,\n\n\n\n\n\n\n\t\t \t\t\t-\n\n\n\n\n\n1\n\n\n\...
3,Die ochtend smeerde moeder ons een voor een i...
4,De deksel van het blik was zo vettig dat je h...


In [6]:
# Save data frame as .csv file for intermediate manual verification steps
df.to_csv('data/DAIO_Deel_I_sentences.csv', encoding='utf8', sep=';')

## 2. Text preprocessing and tokenization

### 2.1. Load the required libraries and packages

In [7]:
# Import libraries and packages
import re # for regular expressions
import gensim # for topic modeling
from gensim.parsing.preprocessing import STOPWORDS # package for stop word processing

import nltk # natural language toolkit
from nltk.tag import PerceptronTagger # part-of-speech tagging package
from nltk.corpus import alpino as alp # dutch corpus for PerceptronTagger
nltk.download('alpino') # download dutch corpus
from nltk.stem import WordNetLemmatizer # for lemmatization
from nltk.corpus import stopwords # package for stop word processing
from nltk.stem.snowball import DutchStemmer # for stemming

import numpy as np # for working with numerical data

import codecs # codec module

import simplemma # for lemmatization

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\isabe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### 2.2. Create stop word list
A stop word list contains words wich are unnecessary/responsible for negative influence on the generation of topics.

In [9]:
# Open stopword list .txt file for Dutch stopwords
stopword_file = open('data/stopwords_nl.txt', 'r')

In [10]:
# Save stop word file as variable
stopwords = stopword_file.read()

In [11]:
# Print stopword list
print(stopwords)

aan,
aangaande,
aangezien,
achte,
achter,
achterna,
af,
afgelopen,
al,
aldaar,
aldus,
alhoewel,
alias,
alle,
allebei,
alleen,
alles,
als,
alsnog,
altijd,
altoos,
ander,
andere,
anders,
anderszins,
beetje,
behalve,
behoudens,
beide,
beiden,
ben,
beneden,
bent,
bepaald,
betreffende,
bij,
bijna,
bijv,
binnen,
binnenin,
blijkbaar,
blijken,
boven,
bovenal,
bovendien,
bovengenoemd,
bovenstaand,
bovenvermeld,
buiten,
bv,
daar,
daardoor,
daarheen,
daarin,
daarna,
daarnet,
daarom,
daarop,
daaruit,
daarvanlangs,
dan,
dat,
de,
deden,
deed,
der,
derde,
derhalve,
dertig,
deze,
dhr,
die,
dikwijls,
dit,
doch,
doe,
doen,
doet,
door,
doorgaand,
drie,
duizend,
dus,
echter,
een,
eens,
eer,
eerdat,
eerder,
eerlang,
eerst,
eerste,
eigen,
eigenlijk,
elk,
elke,
en,
enig,
enige,
enigszins,
enkel,
er,
erdoor,
erg,
ergens,
etc,
etcetera,
even,
eveneens,
evenwel,
gauw,
ge,
gedurende,
geen,
gehad,
gekund,
geleden,
gelijk,
gemoeten,
gemogen,
genoeg,
geweest,
gewoon,
gewoonweg,
haar,
haarzelf,
had,
hadden,
hare,
he

In [12]:
# Split stop word string into a list
stop_words = stopwords.split(',\n')

In [13]:
# Close stopword file
stopword_file.close()

In [14]:
# Display list with stopwords for words wich are unnecessary/responsible for negative influence on the generation of topics
stop_words

['aan',
 'aangaande',
 'aangezien',
 'achte',
 'achter',
 'achterna',
 'af',
 'afgelopen',
 'al',
 'aldaar',
 'aldus',
 'alhoewel',
 'alias',
 'alle',
 'allebei',
 'alleen',
 'alles',
 'als',
 'alsnog',
 'altijd',
 'altoos',
 'ander',
 'andere',
 'anders',
 'anderszins',
 'beetje',
 'behalve',
 'behoudens',
 'beide',
 'beiden',
 'ben',
 'beneden',
 'bent',
 'bepaald',
 'betreffende',
 'bij',
 'bijna',
 'bijv',
 'binnen',
 'binnenin',
 'blijkbaar',
 'blijken',
 'boven',
 'bovenal',
 'bovendien',
 'bovengenoemd',
 'bovenstaand',
 'bovenvermeld',
 'buiten',
 'bv',
 'daar',
 'daardoor',
 'daarheen',
 'daarin',
 'daarna',
 'daarnet',
 'daarom',
 'daarop',
 'daaruit',
 'daarvanlangs',
 'dan',
 'dat',
 'de',
 'deden',
 'deed',
 'der',
 'derde',
 'derhalve',
 'dertig',
 'deze',
 'dhr',
 'die',
 'dikwijls',
 'dit',
 'doch',
 'doe',
 'doen',
 'doet',
 'door',
 'doorgaand',
 'drie',
 'duizend',
 'dus',
 'echter',
 'een',
 'eens',
 'eer',
 'eerdat',
 'eerder',
 'eerlang',
 'eerst',
 'eerste',
 'ei

### 2.2. Stop word removing, lemmatization and tokenization processes

In [16]:
# Loads the fitting lemmatization data for the dutch language
langdata = simplemma.load_data('nl')

In [17]:
# Creates a function to lemmatize a token
def lemmatize(token):
    return simplemma.lemmatize(token, langdata)

In [18]:
# Creates a function to remove stopwords, tokenize and lemmatize the given text
def tokenize(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result



### 2.3. Part-of-speech tagging

In [20]:
# Train the PoS tagger on the dutch "alp" corpus
training_corpus = list(alp.tagged_sents()) 
tagger = PerceptronTagger(load=True)
tagger.train(training_corpus)

In [21]:
# Display classes from the tagger function for possible exclusion of word groups
sorted(tagger.classes)

['#',
 '$',
 "''",
 '(',
 ')',
 ',',
 '.',
 ':',
 'CC',
 'CD',
 'DT',
 'EX',
 'FW',
 'IN',
 'JJ',
 'JJR',
 'JJS',
 'LS',
 'MD',
 'NN',
 'NNP',
 'NNPS',
 'NNS',
 'PDT',
 'POS',
 'PRP',
 'PRP$',
 'RB',
 'RBR',
 'RBS',
 'RP',
 'SYM',
 'TO',
 'UH',
 'VB',
 'VBD',
 'VBG',
 'VBN',
 'VBP',
 'VBZ',
 'WDT',
 'WP',
 'WP$',
 'WRB',
 '``',
 'adj',
 'adv',
 'comp',
 'comparative',
 'det',
 'fixed',
 'name',
 'noun',
 'num',
 'part',
 'pp',
 'prep',
 'pron',
 'punct',
 'tag',
 'verb',
 'vg']

### 2.4. Merging of the created processes

In [ ]:
# Variable 'punctuation' with special characters
punctuation = '!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@'

In [22]:
# Combines the previously functions into one process
# apply the following: lowercase, strip punctuation, extra spaces or numbers in text

def preprocess_text(text):
    text = text.lower()  # lower case
    text = re.sub('[' + punctuation + ']+', ' ', text)  # strip punctuation
    text = re.sub('\s+', ' ', text)  # remove double spacing
    text = re.sub('([0-9]+)', '', text)  # remove numbers
    text_token_list = tokenize(text)  # apply lemmatization and tokenization
    pos_comment = tagger.tag(text_token_list) #apply PoS tagging
    text_pos=[word[0] for word in pos_comment if word[1] in ['noun', 'adj']] # remove every class but nouns and adjectives
    text = ' '.join([w for w in text_pos]) # add remaining words
    return text

In [25]:
# Function to read in, clean, preprocess and tokenize text in dataframe column and save tokens in a new column
def tokenize_text(df):
    df['tokens'] = df.text.apply(preprocess_text) # creates new column and saves the functions output into
    num_texts = len(df)
    print('Complete. Number of text chunks that have been cleaned and tokenized: {}'.format(num_texts))
    return df

In [26]:
# Remove string control characters [\\t, \\n, \\r, \t, \n, \r] from data frame strings
df = df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=False)
df.head()

,text
0,"Er staat geschreven, ‘ik maak alle dingen nieu..."
1,"De ijzelregen geselt bloei tot glazig moes,Een..."
2,-1Ik was tien jaar en deed mijn jas niet meer...
3,Die ochtend smeerde moeder ons een voor een i...
4,De deksel van het blik was zo vettig dat je h...


### 2.5. Application of all functions on the data frame

In [27]:
# Apply tokenize_text on data frame and display cleaned and tokenized number of text chunks
texts_df  = tokenize_text(df)

Complete. Number of text chunks that have been cleaned and tokenized: 376


In [31]:
# .head() outputs the first 5 rows of the created data frame called "texts_df"
texts_df.head()

,text,tokens
0,"Er staat geschreven, ‘ik maak alle dingen nieu...",ding nieuw akkoord waslijn verdriet messcherp ...
1,"De ijzelregen geselt bloei tot glazig moes,Een...",glazig moes hondsvot pels kurkdroog
2,-1Ik was tien jaar en deed mijn jas niet meer...,jaar
3,Die ochtend smeerde moeder ons een voor een i...,ochtend uierzalf vrieskou bogena normaal afspr...
4,De deksel van het blik was zo vettig dat je h...,vettig theedoek roken gaargestoofd uierboord d...


In [33]:
# Save new dataframe as .csv file
texts_df.to_csv('data/DAIO_Deel_I_preprocessed_df_all_noun_adj.csv', index=False, header=True, sep=';', encoding='utf-8')

## 3. Topic modeling

In [34]:
# Import libraries and packages
import spacy # NLP library

import gensim.corpora as corpora # for creating a corpora
from gensim.models import CoherenceModel # for creating the coherence model 

from pprint import pprint # provides capability to "pretty-print" data structures

import pyLDAvis.gensim_models # pyLDAvis gensim models
import pickle # for serializing and de-serializing a object structure
import pyLDAvis # data visualization tool

### 3.1. Preprocess tokens

In [35]:
# Assign new variable to data frame
preprocessed_texts = texts_df

In [36]:
# .head() outputs the first 5 rows of the created data frame called "preprocessed_texts"
preprocessed_texts.head()

,text,tokens
0,"Er staat geschreven, ‘ik maak alle dingen nieu...",ding nieuw akkoord waslijn verdriet messcherp ...
1,"De ijzelregen geselt bloei tot glazig moes,Een...",glazig moes hondsvot pels kurkdroog
2,-1Ik was tien jaar en deed mijn jas niet meer...,jaar
3,Die ochtend smeerde moeder ons een voor een i...,ochtend uierzalf vrieskou bogena normaal afspr...
4,De deksel van het blik was zo vettig dat je h...,vettig theedoek roken gaargestoofd uierboord d...


In [37]:
# Display info of preprocessed df
preprocessed_texts.info

<bound method DataFrame.info of                                                   text  \
0    Er staat geschreven, ‘ik maak alle dingen nieu...   
1    De ijzelregen geselt bloei tot glazig moes,Een...   
2     -1Ik was tien jaar en deed mijn jas niet meer...   
3     Die ochtend smeerde moeder ons een voor een i...   
4     De deksel van het blik was zo vettig dat je h...   
..                                                 ...   
371     Obbe stootte me aan en zei: ‘Daar komt hij aan   
372  ’ Ik tuurde naar de plek waar hij wees en zag ...   
373   Dominee Renkema stapte uit en twee van mijn ooms   
374   Vader had hen en boer Evertsen en Janssen gek...   
375                                                      

                                                tokens  
0    ding nieuw akkoord waslijn verdriet messcherp ...  
1                  glazig moes hondsvot pels kurkdroog  
2                                                 jaar  
3    ochtend uierzalf vrieskou bogena norma

In [38]:
# Create a object which contains the tokens of the preprocessed_texts data frame
texts = preprocessed_texts['tokens']

In [39]:
# .head() outputs the first 5 rows of the object called "texts"
texts.head()

0    ding nieuw akkoord waslijn verdriet messcherp ...
1                  glazig moes hondsvot pels kurkdroog
2                                                 jaar
3    ochtend uierzalf vrieskou bogena normaal afspr...
4    vettig theedoek roken gaargestoofd uierboord d...
Name: tokens, dtype: object

In [40]:
# Save texts object values to list
data = texts.values.tolist()

In [41]:
# Create function to convert data into a list of tokens
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=False, min_len=2, max_len=15))

In [42]:
# Create list of lists
data_words = list(sent_to_words(data))
data_words.head()

In [43]:
# Output data_words
data_words

[['ding',
  'nieuw',
  'akkoord',
  'waslijn',
  'verdriet',
  'messcherp',
  'vlaag',
  'vlucht',
  'wreed'],
 ['glazig', 'moes', 'hondsvot', 'pels', 'kurkdroog'],
 ['jaar'],
 ['ochtend',
  'uierzalf',
  'vrieskou',
  'bogena',
  'normaal',
  'afspreken',
  'kloof',
  'eeltringen',
  'knobbel',
  'speen'],
 ['vettig',
  'theedoek',
  'roken',
  'gaargestoofd',
  'uierboord',
  'dik',
  'snede',
  'besprenkelen',
  'zout',
  'peper',
  'pan',
  'bouillon',
  'fornuis',
  'gruwelen',
  'stinkende',
  'zalven'],
 ['vader', 'dik', 'vinger', 'kloppen', 'korst'],
 ['wang', 'lichten', 'keukenpeertje', 'vliegenkak'],
 ['jaar', 'lampenkap', 'mooi', 'bloem', 'dorp', 'vader'],
 ['jaar'],
 ['kerst',
  'vettig',
  'duimen',
  'oogkas',
  'bang',
  'hard',
  'duwen',
  'oogbal',
  'knikker',
  'rol'],
 ['afdwalen', 'stil', 'goed', 'gelovig', 'godin', 'moment'],
 ['open', 'sneeuwbui', 'dommig'],
 ['ontbijttafel', 'riet', 'vlechten', 'broodmand', 'servet', 'kerstengeltjes'],
 ['trompet', 'maretak', '

### 3.2. Dictionary and corpus for the LDA Model

In [45]:
# Creates dictionary which contains the number of times a word appears in "data_words"
id2word = corpora.Dictionary(data_words)

count = 0
for k, v in id2word.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 akkoord
1 ding
2 messcherp
3 nieuw
4 verdriet
5 vlaag
6 vlucht
7 waslijn
8 wreed
9 glazig
10 hondsvot


In [46]:
# Filter out tokens that appear in less than 5 documents, more than 0.5 documents, and only the top 10000 tokens
id2word.filter_extremes(no_below=5, no_above=0.5, keep_n=10000)

In [47]:
# Create corpus
texts = data_words

In [48]:
# Convert document into bag-of-words format
corpus = [id2word.doc2bow(text) for text in texts]

### 3.3. Create LDA model

In [49]:
# Variable for number of topics
num_topics = 5

In [50]:
# Build LDA model with pre defined corpus and dictionary
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)# Print the Keyword in the topics

In [51]:
# Print the most significant topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.081*"groot" + 0.066*"broer" + 0.066*"wild" + 0.052*"ijs" + 0.052*"donker" '
  '+ 0.045*"obbe" + 0.038*"laag" + 0.031*"moeder" + 0.030*"godin" + '
  '0.030*"worm"'),
 (1,
  '0.098*"hoofd" + 0.060*"vader" + 0.058*"matthies" + 0.058*"elkaar" + '
  '0.044*"badwater" + 0.044*"glas" + 0.043*"hard" + 0.031*"lang" + '
  '0.031*"wang" + 0.031*"lichaam"'),
 (2,
  '0.131*"moeder" + 0.092*"matthies" + 0.048*"vader" + 0.047*"jaar" + '
  '0.047*"koe" + 0.039*"bed" + 0.032*"lang" + 0.032*"rand" + 0.032*"avond" + '
  '0.032*"mooi"'),
 (3,
  '0.113*"moeder" + 0.061*"mens" + 0.052*"lichaam" + 0.044*"kisten" + '
  '0.044*"mond" + 0.036*"lang" + 0.036*"oma" + 0.036*"voet" + 0.036*"land" + '
  '0.028*"vader"'),
 (4,
  '0.205*"vader" + 0.120*"moeder" + 0.047*"dag" + 0.034*"konijn" + '
  '0.034*"engeltje" + 0.034*"servet" + 0.028*"lage" + 0.028*"vinger" + '
  '0.028*"naasten" + 0.021*"matthies"')]


### 3.4. Evaluate LDA model

In [53]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', round(coherence_lda, 2))

Coherence Score:  0.59


### 3.5. Visualize LDA model

In [55]:
# Enable display of prepared model data
pyLDAvis.enable_notebook()

# Sets the file path
LDAvis_data_filepath = os.path.join('results/ldavis_prepared_Deel_I_'+str(num_topics))

In [56]:
# Save the computed LDA data to file path
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

C:\Users\isabe\anaconda3\envs\tm_master_NL\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [57]:
# Load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

In [ ]:
# Save final topic model as html file
pyLDAvis.save_html(LDAvis_prepared, 'results/ldavis_prepared_Deel_I_'+ str(num_topics) +'.html')

In [58]:
# Display the final topic model in a dashboard
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.120510  0.020888       1        1  22.119205
2      0.067511  0.038162       2        1  22.009551
0     -0.126788 -0.072398       3        1  21.620488
3      0.025083 -0.117781       4        1  19.318474
1     -0.086315  0.131130       5        1  14.932282, topic_info=      Term       Freq      Total Category  logprob  loglift
1    vader  39.000000  39.000000  Default  30.0000  30.0000
20   hoofd  11.000000  11.000000  Default  29.0000  29.0000
25  moeder  46.000000  46.000000  Default  28.0000  28.0000
38    mens   7.000000   7.000000  Default  27.0000  27.0000
11   groot  15.000000  15.000000  Default  26.0000  26.0000
..     ...        ...        ...      ...      ...      ...
27   hanna   0.243873   5.252771   Topic5  -5.8103  -1.1682
25  moeder   0.339418  46.065915   Topic5  -5.4797  -3.0090
16  donker   0.259910   8.731485   Topic5  -5.7466  -1.6127
36     koe   0.260781   9.808662   Topic5  -5.7432  -1.7257
11   groot   0.261051  15.457324   Topic5  -5.7422  -2.1795

[210 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
40        2  0.351789    arm
40        3  0.234526    arm
40        4  0.117263    arm
40        5  0.234526    arm
44        1  0.186016  avond
...     ...       ...    ...
55        3  0.776514   worm
23        1  0.190409    één
23        2  0.190409    één
23        3  0.380819    één
23        4  0.190409    één

[191 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 3, 1, 4, 2])